<a href="https://colab.research.google.com/github/ariadna-guzman/E3-Deteccion-de-plagio-TC3002B.301/blob/main/Evidencia_2_Modelo_Mejorado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evidencia 2: Modelo Mejorado

- Ariadna Jocelyn Guzmán Jiménez A01749373
- Jorge Chávez Badillo A01749448
- Amy Murakami Tsutsumi A01750185


## Instalación de librerías

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.8 MB/s eta 0:00:00


In [3]:
!pip -q install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00


In [4]:
!pip -q install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [9]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00


## Importación de librerías

In [16]:
# Kit de herramientas de lenguaje natural
import nltk 
from nltk.stem import PorterStemmer
from nltk.util import ngrams
import torch
# Para el cálculo de la distancia entre párrafos
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Para la manipulación y lectura de archivos
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
# Para la visualización, análisis y manipulación de los datos 
import pandas as pd
from tqdm import tqdm 
# Para la creación de vectores y matrices
import numpy as np
from keras_preprocessing.sequence import pad_sequences
# Implementación de BERT
from transformers import BertTokenizer, AutoModelForSequenceClassification
# Para la traducción de textos
from transformers import MarianMTModel, MarianTokenizer
from langdetect import detect, DetectorFactory

## Importación de Textos

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
genuinos = '/content/drive/Shareddrives/Desarrollo de Aplicaciones Avanzadas de Ciencias Computacionales/Reto/Evidencia 2/2023-Datasets/Etapa1/documentos-genuinos/'
sospechosos = '/content/drive/Shareddrives/Desarrollo de Aplicaciones Avanzadas de Ciencias Computacionales/Reto/Evidencia 2/2023-Datasets/Etapa1/docmentos-sospechosos/'

### Creación de data frame

In [20]:
def create_frame(path):
  content = []
  names = []
  files_list = sorted(os.listdir(path))
  for file_name in files_list:
    with open(path + file_name, 'r') as file:
      data = file.read().rstrip()
      content.append(data)
      names.append(file_name)
  dictionary = {'name': names, 'content': content}
  df = pd.DataFrame(dictionary)
  return df

In [22]:
create_frame(genuinos).head()

,name,content
0,org-300.txt,Automatic software plagiarism detection tools ...
1,org-301.txt,A casual comment by a student alerted the auth...
2,org-302.txt,Paraphrase types have been proposed by researc...
3,org-303.txt,This paper addresses the issue of text matchin...
4,org-304.txt,This work presents a Sentence Hashing Algorith...


## Funciones para el Análisis de Similitud

### Lemmatizer

In [ ]:
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
#Función para aplicar lematización en los párrafos
def lemm_parrafo(parrafo):
  lemmatizer = WordNetLemmatizer()
  parrafo_lemm = []

  for palabra in parrafo: 
    palabra_lemm = lemmatizer.lemmatize(palabra)
    parrafo_lemm.append(palabra_lemm)

  parrafo_str = ' '.join(parrafo_lemm)
  return parrafo_str

### Stemming

In [ ]:
# Función para aplicar la herramienta de stemming a los párrafos
def stemm_parrafo(parrafo):
  stemmer = PorterStemmer()
  parrafo = parrafo.split()
  parrafo_stemm = []

  for palabra in parrafo:
    palabra_stem = stemmer.stem(palabra)
    parrafo_stemm.append(palabra_stem)

  parrafo_str = ' '.join(parrafo_stemm)
  return parrafo_str

## Conclusiones